In [2]:
# Downgrade pip to avoid metadata validation issues
!pip install pip==23.2.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 35.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [3]:
!pip install transformers fairseq nltk


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 22.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Obtaining dependency information for hydra-core<1.1,>=1.0.7 from https://files.pythonhosted.org/packages/5f/2a/9c698daa12ed6e09e7629e6908528f043fa9de8a441c56cc13608d765fb2/hydra_core-1.0.7-py3-none-any.whl.metadata
  Obtaining dependency information for omegaconf<2.1 from https://files.pythonhosted.org/packages/d0/eb/9d63ce09dd8aa85767c65668d5414958ea29648a0eec80a4a7d311ec2684/omegaconf-2.0.6-py3-none-any.whl.metadata
  Obtaining dependency information for sacrebleu>=1.4.12 from https://files.pythonhosted.org/packages/cd/45/7b55a7bd7e5c5b573b40ad58ba43fa09962dc5c8d71b1f573d4aeaa54a7e/sacrebleu-2.5.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 8.0 MB/s eta 0:00:00
  Obtaining dependency infor

In [9]:
!pip install transformers torch spacy requests

DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


# mGENRE (Wikidata)

## This only finds entities

In [53]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import spacy

# Load spaCy NER model for entity detection
nlp = spacy.load("en_core_web_sm")

# Load GENRE model and tokenizer
model_name = "facebook/genre-linking-aidayago2"  # You can use any available GENRE model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Full text to process
text = """Barack Obama, the 44th President of the United States, was born in Honolulu, Hawaii.
He served two terms in office from 2009 to 2017.
During his presidency, Obama signed the Affordable Care Act into law,
a significant reform of the healthcare system. In 2015, Obama visited Cuba,
marking the first time a sitting U.S. president had visited the island in nearly 90 years.
After leaving office, he continued his work on global issues and humanitarian causes."""

# Process raw text with spaCy NER to get entities
doc = nlp(text)
entities = [ent.text for ent in doc.ents]

# Perform entity linking with GENRE for each detected entity
for entity in entities:
    # Link entity using GENRE (mGENRE will generate a valid entity name from input text)
    inputs = tokenizer([f"[START] {entity} [END]"], return_tensors="pt")  # You can skip [START]/[END] for GENRE
    outputs = model.generate(**inputs)
    result = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # Print the result
    print(f"Entity: {entity} → Linked to: {result}")


Entity: Barack Obama → Linked to: Barack Obama
Entity: 44th → Linked to: 44th United States Congress
Entity: the United States → Linked to: United States
Entity: Honolulu → Linked to: Honolulu
Entity: Hawaii → Linked to: Hawaii
Entity: two → Linked to: 2
Entity: 2009 → Linked to: 2009 in paleontology
Entity: Obama → Linked to: Barack Obama
Entity: the Affordable Care Act → Linked to: Patient Protection and Affordable Care Act
Entity: 2015 → Linked to: 2015 in paleontology
Entity: Obama → Linked to: Barack Obama
Entity: Cuba → Linked to: Cuba
Entity: first → Linked to: First
Entity: U.S. → Linked to: United States
Entity: nearly 90 years → Linked to: Bertram Bingham III


## This code finds the Wikipedia URL.

In [54]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import spacy

# Load spaCy NER model for entity detection
nlp = spacy.load("en_core_web_sm")

# Load GENRE model and tokenizer
model_name = "facebook/genre-linking-aidayago2"  # You can use any available GENRE model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# # Full text to process
# text = """Barack Obama was born in Hawaii. He served as the 44th President of the United States. Hawaii is a state in the Pacific Ocean and in Iran. Hassan Rouhani was an Iranian president."""
# Full text to process
text = """Barack Obama, the 44th President of the United States, was born in Honolulu, Hawaii.
He served two terms in office from 2009 to 2017.
During his presidency, Obama signed the Affordable Care Act into law,
a significant reform of the healthcare system. In 2015, Obama visited Cuba,
marking the first time a sitting U.S. president had visited the island in nearly 90 years.
After leaving office, he continued his work on global issues and humanitarian causes."""

# Process raw text with spaCy NER to get entities
doc = nlp(text)
entities = [ent.text for ent in doc.ents]

# Function to extract the Wikidata URL from the GENRE model output
def get_wikidata_url(entity_name):
    # Construct the Wikidata URL using the entity name (i.e., QID)
    url = f"https://en.wikipedia.org/wiki/{entity_name.replace(' ', '_')}"
    response = requests.get(url)

    if response.status_code == 200:
        return url
    else:
        return f"https://en.wikipedia.org/wiki/Q{entity_name}"  # Fallback for QID


# Process each sentence and link entities
for sentence in entities:
    # Link entity using GENRE
    inputs = tokenizer([f"[START] {sentence} [END]"], return_tensors="pt")
    outputs = model.generate(**inputs)
    result = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    # print(result)
    # Extract the Wikidata URL from the result
    wikidata_url = get_wikidata_url(result)

    if wikidata_url:
        print(f"Entity: {sentence} → Linked to: {wikidata_url}")
    else:
        print(f"Entity: {sentence} could not be linked to a valid Wikidata URL.")


Entity: Barack Obama → Linked to: https://en.wikipedia.org/wiki/Barack_Obama
Entity: 44th → Linked to: https://en.wikipedia.org/wiki/44th_United_States_Congress
Entity: the United States → Linked to: https://en.wikipedia.org/wiki/United_States
Entity: Honolulu → Linked to: https://en.wikipedia.org/wiki/Honolulu
Entity: Hawaii → Linked to: https://en.wikipedia.org/wiki/Hawaii
Entity: two → Linked to: https://en.wikipedia.org/wiki/2
Entity: 2009 → Linked to: https://en.wikipedia.org/wiki/2009_in_paleontology
Entity: Obama → Linked to: https://en.wikipedia.org/wiki/Barack_Obama
Entity: the Affordable Care Act → Linked to: https://en.wikipedia.org/wiki/Patient_Protection_and_Affordable_Care_Act
Entity: 2015 → Linked to: https://en.wikipedia.org/wiki/2015_in_paleontology
Entity: Obama → Linked to: https://en.wikipedia.org/wiki/Barack_Obama
Entity: Cuba → Linked to: https://en.wikipedia.org/wiki/Cuba
Entity: first → Linked to: https://en.wikipedia.org/wiki/First
Entity: U.S. → Linked to: htt

# Linking to Wikidata (Spacy Only)

## One Sentence Matching

In [24]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import spacy

# Load spaCy NER model
nlp = spacy.load("en_core_web_sm")

# Load mGENRE model and tokenizer
model_name = "facebook/mgenre-wiki"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Raw text to process
text = "Barack Obama was born in Hawaii and served as the 44th President of the United States."

# Detect entities using spaCy
doc = nlp(text)
entities = [ent.text for ent in doc.ents]

# Define a function to get QID from Wikidata for each entity
def get_qid(entity_name):
    url = f"https://www.wikidata.org/w/api.php"
    params = {
        "action": "wbsearchentities",
        "search": entity_name,
        "language": "en",
        "limit": 1,
        "format": "json"
    }
    response = requests.get(url, params=params).json()

    if "search" in response and len(response["search"]) > 0:
        qid = response["search"][0]["id"]
        return f"http://www.wikidata.org/entity/{qid}"
    else:
        return None  # Return None if no matching entity is found

# Link each entity to its Wikidata QID, removing any language tags
for entity in entities:
    # Strip language code (e.g., ' >> en') and use only the entity name
    entity_name = entity.split(" >>")[0].strip()
    if entity_name:  # Ensure that entity name is not empty
        qid = get_qid(entity_name)
        if qid:
            print(f"Entity: {entity_name} → Linked to QID: {qid}")
        else:
            print(f"Entity: {entity_name} could not be linked to a QID.")


Entity: Barack Obama → Linked to QID: http://www.wikidata.org/entity/Q76
Entity: Hawaii → Linked to QID: http://www.wikidata.org/entity/Q782
Entity: 44th → Linked to QID: http://www.wikidata.org/entity/Q21084461
Entity: the United States → Linked to QID: http://www.wikidata.org/entity/Q30


## Several Sentences Matching (Text)

In [34]:
import spacy
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import requests

# Load spaCy NER model
nlp = spacy.load("en_core_web_sm")

# Load mGENRE model and tokenizer
model_name = "facebook/mgenre-wiki"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Full text to process
text = """Barack Obama was born in Hawaii. He served as the 44th President of the United States. Hawaii is a state in the Pacific Ocean and in Iran. Hassan Rouhani was an Iranian president."""

# Function to get Wikidata QID
def get_qid(entity_name):
    url = f"https://www.wikidata.org/w/api.php"
    params = {
        "action": "wbsearchentities",
        "search": entity_name,
        "language": "en",
        "limit": 1,
        "format": "json"
    }
    response = requests.get(url, params=params).json()

    if "search" in response and len(response["search"]) > 0:
        qid = response["search"][0]["id"]
        return f"http://www.wikidata.org/entity/{qid}"
    else:
        return None  # Return None if no matching entity is found

# Split the text into sentences
doc = nlp(text)
sentences = [sent.text for sent in doc.sents]

# Process each sentence
for sentence in sentences:
    print(f"\nProcessing sentence: {sentence}")

    # Detect entities using spaCy
    doc = nlp(sentence)
    entities = [ent.text for ent in doc.ents]

    # Link each entity to its Wikidata QID
    for entity in entities:
        # Strip language code (e.g., ' >> en') and use only the entity name
        entity_name = entity.split(" >>")[0].strip()
        if entity_name:  # Ensure that entity name is not empty
            qid = get_qid(entity_name)
            if qid:
                print(f"Entity: {entity_name} → Linked to QID: {qid}")
            else:
                print(f"Entity: {entity_name} could not be linked to a QID.")



Processing sentence: Barack Obama was born in Hawaii.
Entity: Barack Obama → Linked to QID: http://www.wikidata.org/entity/Q76
Entity: Hawaii → Linked to QID: http://www.wikidata.org/entity/Q782

Processing sentence: He served as the 44th President of the United States.
Entity: 44th → Linked to QID: http://www.wikidata.org/entity/Q21084461
Entity: the United States → Linked to QID: http://www.wikidata.org/entity/Q30

Processing sentence: Hawaii is a state in the Pacific Ocean and in Iran.
Entity: Hawaii → Linked to QID: http://www.wikidata.org/entity/Q782
Entity: the Pacific Ocean → Linked to QID: http://www.wikidata.org/entity/Q7755858
Entity: Iran → Linked to QID: http://www.wikidata.org/entity/Q794

Processing sentence: Hassan Rouhani was an Iranian president.
Entity: Hassan Rouhani → Linked to QID: http://www.wikidata.org/entity/Q348144
Entity: Iranian → Linked to QID: http://www.wikidata.org/entity/Q9168


In [40]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import spacy
import requests

# Load spaCy NER model
nlp = spacy.load("en_core_web_sm")

# Load mGENRE model and tokenizer
model_name = "facebook/mgenre-wiki"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Full text to process
text = """Barack Obama was born in Hawaii. He served as the 44th President of the United States. Hawaii is a state in the Pacific Ocean and in Iran. Hassan Rouhani was an Iranian president."""

# Function to get QID from mGENRE model
def link_entity_mgenre(entity_name):
    # Use mGENRE to link the entity (replace with your mGENRE entity-linking logic)
    inputs = tokenizer([f"[START] {entity_name} [END]"], return_tensors="pt")
    outputs = model.generate(**inputs)
    result = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    if ">>" in result:
        return f"http://www.wikidata.org/entity/{result.split(' >> ')[1]}"
    return None

# Split the text into sentences
doc = nlp(text)
sentences = [sent.text for sent in doc.sents]

# Process each sentence
for sentence in sentences:
    print(f"\nProcessing sentence: {sentence}")

    # Detect entities using spaCy
    doc = nlp(sentence)
    entities = [ent.text for ent in doc.ents]

    # Link each entity to its Wikidata QID using mGENRE
    for entity in entities:
        # Link the entity to a QID using mGENRE
        qid = link_entity_mgenre(entity)
        if qid:
            print(f"Entity: {entity} → Linked to QID: {qid}")
        else:
            print(f"Entity: {entity} could not be linked to a QID.")



Processing sentence: Barack Obama was born in Hawaii.
Entity: Barack Obama → Linked to QID: http://www.wikidata.org/entity/en
Entity: Hawaii → Linked to QID: http://www.wikidata.org/entity/it

Processing sentence: He served as the 44th President of the United States.
Entity: 44th → Linked to QID: http://www.wikidata.org/entity/en
Entity: the United States → Linked to QID: http://www.wikidata.org/entity/en

Processing sentence: Hawaii is a state in the Pacific Ocean and in Iran.
Entity: Hawaii → Linked to QID: http://www.wikidata.org/entity/it
Entity: the Pacific Ocean → Linked to QID: http://www.wikidata.org/entity/en
Entity: Iran → Linked to QID: http://www.wikidata.org/entity/en

Processing sentence: Hassan Rouhani was an Iranian president.
Entity: Hassan Rouhani → Linked to QID: http://www.wikidata.org/entity/en
Entity: Iranian → Linked to QID: http://www.wikidata.org/entity/en


# Wikipedia

In [39]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import spacy
import requests

# Load spaCy NER model
nlp = spacy.load("en_core_web_sm")

# Load mGENRE model and tokenizer
model_name = "facebook/mgenre-wiki"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Raw text to process
text = """Barack Obama was born in Hawaii. He served as the 44th President of the United States. Hawaii is a state in the Pacific Ocean and in Iran. Hassan Rouhani was an Iranian president."""

# Detect entities using spaCy
doc = nlp(text)
entities = [ent.text for ent in doc.ents]

# Define a function to get Wikipedia URL from entity name or QID
def get_wikipedia_url(entity_name):
    # First check if the entity is in Wikipedia
    url = f"https://en.wikipedia.org/wiki/{entity_name.replace(' ', '_')}"
    response = requests.get(url)

    if response.status_code == 200:
        return url
    else:
        return f"https://en.wikipedia.org/wiki/Q{entity_name}"  # Fallback for QID

# Process each entity with mGENRE to link to Wikidata QID and Wikipedia URL
for entity in entities:
    # Link entity using mGENRE
    inputs = tokenizer([f"[START] {entity} [END]"], return_tensors="pt")
    outputs = model.generate(**inputs)
    result = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # Ensure the output is a valid Wikidata entity
    if ">>" in result:
        entity_name = result.split(" >>")[0].strip()
        wikipedia_url = get_wikipedia_url(entity_name)
        print(f"Entity: {entity_name} → Wikipedia URL: {wikipedia_url}")
    else:
        print(f"Entity: {entity} could not be linked to a Wikipedia URL.")


Entity: Barack Obama → Wikipedia URL: https://en.wikipedia.org/wiki/Barack_Obama
Entity: Hawaii → Wikipedia URL: https://en.wikipedia.org/wiki/Hawaii
Entity: 44th Division → Wikipedia URL: https://en.wikipedia.org/wiki/44th_Division
Entity: United States → Wikipedia URL: https://en.wikipedia.org/wiki/United_States
Entity: Hawaii → Wikipedia URL: https://en.wikipedia.org/wiki/Hawaii
Entity: Pacific Ocean → Wikipedia URL: https://en.wikipedia.org/wiki/Pacific_Ocean
Entity: Iran → Wikipedia URL: https://en.wikipedia.org/wiki/Iran
Entity: Hassan Rouhani → Wikipedia URL: https://en.wikipedia.org/wiki/Hassan_Rouhani
Entity: Iranian peoples → Wikipedia URL: https://en.wikipedia.org/wiki/Iranian_peoples


# Linking to DBPedia

In [30]:
!pip install fuzzywuzzy

  Obtaining dependency information for fuzzywuzzy from https://files.pythonhosted.org/packages/43/ff/74f23998ad2f93b945c0309f825be92e04e0348e062026998b5eefef4c33/fuzzywuzzy-0.18.0-py2.py3-none-any.whl.metadata
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [36]:
import spacy
import requests

# Load spaCy NER model
nlp = spacy.load("en_core_web_sm")

# Function to get DBpedia URI for an entity
def get_dbpedia_uri(entity_name):
    # Define the SPARQL endpoint for DBpedia
    endpoint = "https://dbpedia.org/sparql"

    # Define the SPARQL query to search for the entity in DBpedia
    query = f"""
    SELECT ?resource
    WHERE {{
        ?resource rdfs:label "{entity_name}"@en.
    }}
    LIMIT 1
    """

    # Send the request to the SPARQL endpoint
    params = {
        "query": query,
        "format": "json"
    }
    response = requests.get(endpoint, params=params)

    # Parse the response and return the URI if found
    if response.status_code == 200:
        data = response.json()
        if data["results"]["bindings"]:
            # Extract the URI from the response
            dbpedia_uri = data["results"]["bindings"][0]["resource"]["value"]
            return dbpedia_uri
    return None

# Full text to process
text = """Barack Obama was born in Hawaii. He served as the 44th President of the United States. Hawaii is a state in the Pacific Ocean and Iran."""

# Split the text into sentences
doc = nlp(text)
sentences = [sent.text for sent in doc.sents]

# Process each sentence
for sentence in sentences:
    print(f"\nProcessing sentence: {sentence}")

    # Detect entities using spaCy
    doc = nlp(sentence)
    entities = [ent.text for ent in doc.ents]

    # Link each entity to its DBpedia URI
    for entity in entities:
        # Strip any unwanted characters and use only the entity name
        entity_name = entity.strip()
        if entity_name:  # Ensure that entity name is not empty
            dbpedia_uri = get_dbpedia_uri(entity_name)
            if dbpedia_uri:
                print(f"Entity: {entity_name} → Linked to DBpedia URI: {dbpedia_uri}")
            else:
                print(f"Entity: {entity_name} could not be linked to a DBpedia URI.")



Processing sentence: Barack Obama was born in Hawaii.
Entity: Barack Obama → Linked to DBpedia URI: http://dbpedia.org/resource/Barack_Obama
Entity: Hawaii → Linked to DBpedia URI: http://dbpedia.org/resource/Category:Hawaii

Processing sentence: He served as the 44th President of the United States.
Entity: 44th could not be linked to a DBpedia URI.
Entity: the United States could not be linked to a DBpedia URI.

Processing sentence: Hawaii is a state in the Pacific Ocean and Iran.
Entity: Hawaii → Linked to DBpedia URI: http://dbpedia.org/resource/Category:Hawaii
Entity: the Pacific Ocean could not be linked to a DBpedia URI.
Entity: Iran → Linked to DBpedia URI: http://dbpedia.org/resource/Category:Iran


### The rest

In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load model and tokenizer
model_name = "facebook/mgenre-wiki"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, from_tf=True)

# Sample text (with multiple entities)
text = "[START] Barack Obama [END] was born in [START] Hawaii [END]."

# Split the text into individual entity mentions
mentions = ["[START] Barack Obama [END]", "[START] Hawaii [END]"]

# Process each mention
for mention in mentions:
    # Tokenize and run model
    inputs = tokenizer([mention], return_tensors="pt")
    outputs = model.generate(**inputs)

    # Decode and print result
    result = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    print(f"{mention} → {result}")


All TF 2.0 model weights were used when initializing MBartForConditionalGeneration.

Some weights of MBartForConditionalGeneration were not initialized from the TF 2.0 model and are newly initialized: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[START] Barack Obama [END] → ['Barack Obama >> en']
[START] Hawaii [END] → ['Hawaii >> it']
